- 지금까지의 방법은 two-stage

## region proposals
- 관심있을 만한 영역을 추천하는 방식
- 영역이 무엇인지 예측하는 것이 R-CNN 
    - convolution은 할 수 있지만 크기 문제

<br>

- 최근에는 single stage 방법이 two stage를 거의 따라잡음
- faster-RCNN은 1초 2장, yolo는 1초 35장
- single stage는 일정 크기 이하는 잘 찾지 못함

## bounding box Regressor
- 박스의 차이를 linear regression으로 함.

- Fast R-CNN부터 multi task 방식을 사용하기 시작함
    - 일반적인 loss를 사용하는 건 잘못됨
    - 특징과 영역이 convolution으로 인해 잘 나옴
- 여전히 selective search사용

## Faster R-CNN
- selective 빼고 neural network사용
- **anchor box** 이용
    - 다양하게 참조하는 pyramid?
    
Faster R-CNN: Towards Real-Time Object
Detection with Region Proposal Networks

### RPN
- sliding window를 하는데 가장 있을만한 size로 9개 형태 가정
- 256 -> ZFNet
- 9개 각각에 대해 배경인지 아닌지 판단 -> 좌표값 4k개
    - 특정 모양 영역이 어느 정도 겹치면 물체가 있다고 가정(region proposal)
- k개의 anchor box
- 학습시킬 때 feature map에서 배경인지 아닌지
- YOLO의 grid방식과 비슷
    

- kaggle
https://www.kaggle.com/kishor1210/train-faster-rcnn-using-keras

In [7]:
import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle
import math
import cv2
import copy
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
import os

from sklearn.metrics import average_precision_score

from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, TimeDistributed

from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers

In [8]:
class Config:

    def __init__(self):

        # Print the process or not
        self.verbose = True

        # Name of base network
        self.network = 'vgg'

        # Setting for data augmentation
        self.use_horizontal_flips = False
        self.use_vertical_flips = False
        self.rot_90 = False

        # Anchor box scales
    # Note that if im_size is smaller, anchor_box_scales should be scaled
    # Original anchor_box_scales in the paper is [128, 256, 512]
        self.anchor_box_scales = [64, 128, 256] 

        # Anchor box ratios
        self.anchor_box_ratios = [[1, 1], [1./math.sqrt(2), 2./math.sqrt(2)], [2./math.sqrt(2), 1./math.sqrt(2)]]

        # Size to resize the smallest side of the image
        # Original setting in paper is 600. Set to 300 in here to save training time
        self.im_size = 300

        # image channel-wise mean to subtract
        self.img_channel_mean = [103.939, 116.779, 123.68]
        self.img_scaling_factor = 1.0

        # number of ROIs at once
        self.num_rois = 4

        # stride at the RPN (this depends on the network configuration)
        self.rpn_stride = 16

        self.balanced_classes = False

        # scaling the stdev
        self.std_scaling = 4.0
        self.classifier_regr_std = [8.0, 8.0, 4.0, 4.0]

        # overlaps for RPN
        self.rpn_min_overlap = 0.3
        self.rpn_max_overlap = 0.7

        # overlaps for classifier ROIs
        self.classifier_min_overlap = 0.1
        self.classifier_max_overlap = 0.5

        # placeholder for the class mapping, automatically generated by the parser
        self.class_mapping = None

        self.model_path = None

5개 함수 구현하는 게 customizing의 핵심

In [12]:
import tensorflow as tf
from skimage.io import imread
model = tf.keras.models.Sequential([
    tf.keras.layers.GlobalAvgPool2D()
])
im = imread('1.jpg')
model(im[np.newaxis])

<tf.Tensor: shape=(1, 3), dtype=uint8, numpy=array([[171, 167, 171]], dtype=uint8)>

In [11]:
im[np.newaxis].shape

(1, 1350, 900, 3)

In [ ]:
tf.keras.applications.